# parsing_tir

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display


импорт библиотек

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [3]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

создаем функцию предварительного просмотра данных

url = ['https://tiraet.com/catalog/PHoto--i-video/?PAGEN_1=']

url = ['https://tiraet.com/catalog/bytovaya/?PAGEN_1=102'] 

In [4]:
oll_url=['https://tiraet.com/catalog/Komplektujuschie/?PAGEN_1=',
         'https://tiraet.com/catalog/2873/?PAGEN_1=',
         'https://tiraet.com/catalog/Kompjuternaja-mebel/?PAGEN_1=',
         'https://tiraet.com/catalog/bytovaya/?PAGEN_1=',
         'https://tiraet.com/catalog/ohrannye-sistemy/?PAGEN_1=',
         'https://tiraet.com/catalog/svet-i-jelektrika/?PAGEN_1=',
         'https://tiraet.com/catalog/orgtehnika/?PAGEN_1=',
         'https://tiraet.com/catalog/PHoto--i-video/?PAGEN_1=',
         'https://tiraet.com/catalog/setevoe-oborudovanie/?PAGEN_1=',
         'https://tiraet.com/catalog/jelektropitanie/?PAGEN_1=',
         'https://tiraet.com/catalog/uslugi/?PAGEN_1=',
         'https://tiraet.com/catalog/avtojelektronika/?PAGEN_1=',
         'https://tiraet.com/catalog/rashodnye-materialy/?PAGEN_1=',
         
]

In [5]:
j=1
def parsing_2(url):
    data = pd.DataFrame(columns=['title', 'url', 'date', 'availability', 'id', 'price', 'old_price'])    
    for u in url:
        reqx = requests.get(u)
        soup = bs(reqx.text, 'html.parser')
        max_page = soup.findAll('div', class_='nums')
        maxx = int(max_page[0].text.strip().split('\n')[-1])
        for j in tqdm(range(1,maxx + 1)):        
            reqx = requests.get(u + str(j))
            soup = bs(reqx.text, 'html.parser')
            element = soup.findAll('div', class_='item_info')
            datax = []
            for i in element:
                datax.append({'title' : i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm').text.strip(),
                             'url' : 'https://tiraet.com' + i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm')['href'],
                             'date' : pd.to_datetime('today').normalize(),
                             'availability' : i.find('div', class_='item-stock js-show-stores js-show-info-block').text.strip(),
                             'id' : int(i.find('div', class_='article_block').text.replace('Арт.: ','')),
                             'price' : float(i.find('div', class_='price_matrix_wrapper').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper'))) != "<class 'NoneType'>" else np.NaN ,
                             'old_price' : float(i.find('div', class_='price_matrix_wrapper strike_block').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper strike_block'))) != "<class 'NoneType'>" else np.NaN 
                            })
            datax = pd.DataFrame(datax)
            data = pd.concat([data, datax] , ignore_index=True)
    return data

179  -  16  -  23  -  100  -  33  -  60  -  10  -  4  -  31  -  25  -  5  -  5  -  81 

In [6]:
df = parsing_2(oll_url)
df

  0%|          | 0/187 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

,title,url,date,availability,id,price,old_price
0,"Стойка коммутацион.Netrack (NP5104) 19', 4U 482*177*530mm, black",https://tiraet.com/catalog/Komplektujuschie/komplektujuschie/Korpusa/stoyka_kommutatsion_netrack_np5104_19_4u_482_177_530mm_black/,2024-06-28,Достаточно,87094,NaN,NaN
1,Переходник для мыши COM в PS/2 (A-235),https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/kabelja-udliniteli-i-perehodniki/Perehodnik-dlja-myshi-COM-v-PS2-A-235/,2024-06-28,Много,284,5.0,NaN
2,"Кабель MINI USB2.0 (A - MINI USB 4PM)1,8м (CC-USB2-AM4P-6)",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/kabelja-udliniteli-i-perehodniki/kabel-mini-usb20-a---mini-usb-4pm18m-cc-usb2-am4p-6/,2024-06-28,Много,8683,5.0,NaN
3,"BOX для CD/DVD Verbatim Slim, на 1 диск 120мм, PVC (49979) Black 1шт",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/sumki-dlja-diskov/box_dlya_cd_dvd_verbatim_slim_na_1_disk_120mm_pvc_49979_black_1sht/,2024-06-28,Много,85078,6.6,NaN
4,"Диск CD-R 700MB VERBATIM, 52X (1шт) EXTRA PROTECTION(43411/43351/43437)",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/diski/disk_cdr_700mb_verbatim_52x_1sht_extra_protection_43411_43351_43437/,2024-06-28,Много,12819,6.6,NaN
...,...,...,...,...,...,...,...
11670,Головка для плоттеров Canon PF-03 (2251B001),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/golovka-dlja-plotterov-canon-pf-03-2251b001/,2024-06-28,Мало,27477,7920.0,NaN
11671,Узел термозакрепления (для сетевых апп)CANON iR2202/2204(FM1-F162-0XX),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/uzel_termozakrepleniya_dlya_setevykh_app_canon_ir2202_2204_fm1_f162_0xx/,2024-06-28,Мало,55309,8085.0,NaN
11672,Головка для плоттеров Canon PF-04 (3630B001),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/golovka_dlya_plotterov_canon_pf_04_3630b001/,2024-06-28,Мало,56144,8250.0,NaN
11673,Картридж EPSON MX-80/LX-300/400/800/850/880 Patron,https://tiraet.com/catalog/rashodnye-materialy/Rashodnye-k-matrichnym-printeram-kartridzhi-lenty/kartridji_matr/kartridzh-epson-lx-300400800850880mx-80/,2024-06-28,Нет в наличии,249,38.0,NaN


df.to_csv('csv/tiraet_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

df = pd.read_csv('csv/tiraet_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

In [7]:
tiraet_group = pd.read_csv('csv/atiraet_group.csv')

In [8]:
df = df.merge(tiraet_group, how='left', on='id')

In [9]:
df_new = df[(df['_category'].isna()) | (df['_subcategory'].isna()) | (df['_group'].isna())]

In [10]:
df = df[~((df['_category'].isna()) | (df['_subcategory'].isna()) | (df['_group'].isna()))]

In [11]:
def find_groups(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    elem_1 = soup.findAll('a', class_='breadcrumbs__link colored_theme_hover_bg-el-svg')
    elem_2 = soup.findAll('div', class_='breadcrumbs__item')
    try:
        if len(elem_1) <=2:
            if len(elem_2) <= 4:
                d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_16 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
            else:
                d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_16 = 'T_'+(elem_2[4].text.replace(',','').replace(' ','_').replace('-','_').lower())
        else:
            d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
            d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
            d_16 = 'T_'+(elem_1[2].text.replace(',','').replace(' ','_').replace('-','_').lower())
    except:
        d_14 = 'err_'
        d_15 = 'err_'
        d_16 = 'err_'
        
    return pd.Series([ (d_14), (d_15), (d_16)])

In [12]:
tqdm.pandas() 
df_new[['_category','_subcategory','_group']] = df_new['url'].progress_apply(find_groups)

  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
df_new._category.unique()

array(['T_мобильные_телефоны', 'err_'], dtype=object)

In [14]:
df_new._subcategory.unique()

array(['T_смартфоны', 'err_'], dtype=object)

In [15]:
df_new._group.unique()

array(['T_смартфоны', 'err_'], dtype=object)

In [16]:
data = df_new[~(df_new['_category']=='err_')].reset_index(drop=True)
data = data[['id', '_category','_subcategory','_group' ]]
data = data.sort_values(['_category','_subcategory', '_group'])

In [17]:
tiraet_group = pd.concat([tiraet_group, data], ignore_index=True).sort_values(['_category','_subcategory', '_group']).reset_index(drop=True)
tiraet_group.to_csv('csv/atiraet_group.csv', index=False)  

In [18]:
df = pd.concat([df, df_new], ignore_index=True).sort_values(['_category','_subcategory', '_group']).reset_index(drop=True)

df.to_csv('csv/tiraet_final_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

In [19]:
df[df['_category']=='err_'].id.count()

1

In [20]:
df[df['_category']=='err_']

,title,url,date,availability,id,price,old_price,_category,_subcategory,_group
11678,"Музыкальный центр PANASONIC SC-AKX320GSK (2.0, 450Вт, CD, FM, USB, SD, BT, Д/У) BLACK",https://tiraet.com/catalog/bytovaya/melkaja-bytovaja-tehnika/muzykalnye-centry/muzykalnyy_tsentr_panasonic_sc_akx320gsk_2_0_450vt_cd_fm_usb_sd_bt_d_u_black/,2024-06-28,Мало,87154,3799.0,NaN,err_,err_,err_


In [21]:
df['error_group'] = 0
df.loc[df['_category']=='err', 'error_group'] = 1

In [22]:
df

,title,url,date,availability,id,price,old_price,_category,_subcategory,_group,error_group
0,"Автомагнитола Sencor SCT 3018MR (FM/AM, CD, MP3/WMA, USB, AUX, 4*40W, ISO/RCA, 1DIN)",https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_sencor_sct_3018mr_fm_am_cd_mp3_wma_usb_aux_4_40w_iso_rca_1din/,2024-06-28,Мало,78604,660.0,NaN,T_автотовары,T_автомагнитола,T_автомагнитола,0
1,"Автомагнитола Kenwood KDC-130UG (FM/AM, MP3/WAV/WMA/FLAC, CD, USB, AUX, 4*50W, ISO/RCA, 1DIN) black",https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_kenwood_kdc_130ug_fm_am_mp3_wav_wma_flac_cd_usb_aux_4_50w_iso_rca_1din_black/,2024-06-28,Мало,78235,1419.0,NaN,T_автотовары,T_автомагнитола,T_автомагнитола,0
2,"Автомагнитола Kenwood KMM-BT209 (FM/AM, BT, USB, AUX, RCA, 4*50W, ISO, 1DIN)",https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_kenwood_kmm_bt209_fm_am_bt_usb_aux_rca_4_50w_iso_1din/,2024-06-28,Достаточно,80185,1735.0,NaN,T_автотовары,T_автомагнитола,T_автомагнитола,0
3,"Автомагнитола JVC KDX-382BT (FM/AM, Bluetooth, USB, 4*50W, ISO, AUX, 1DIN)",https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_jvc_kdx_382bt_fm_am_bluetooth_usb_4_50w_iso_aux_1din/,2024-06-28,Мало,80184,1815.0,NaN,T_автотовары,T_автомагнитола,T_автомагнитола,0
4,"Автомагнитола JVC KDT-702BT (FM/AM, Bluetooth, CD, USB, 4*50W, 1DIN)",https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_jvc_kdt_702bt_fm_am_bluetooth_cd_usb_4_50w_1din/,2024-06-28,Достаточно,80183,2099.0,NaN,T_автотовары,T_автомагнитола,T_автомагнитола,0
...,...,...,...,...,...,...,...,...,...,...,...
11674,"Аккумулятор Panasonic ENELOOP (BK-3MCDE/4BE) AA, 2000mAh, Ni-MH, 1.2V 4шт/уп. (214421)",https://tiraet.com/catalog/jelektropitanie/batarei-akkumuljatornye/jelementy-pitanija/akkumulyator_panasonic_eneloop_bk_3mcde_4be_aa_2000mah_ni_mh_1_2v_4sht_up_214421/,2024-06-28,Достаточно,78714,260.0,330.0,T_электропитание,T_элементы_питания,T_батарейки_аккумуляторы,0
11675,Элемент питания Duracell BASIC AA/LR6/MN1500 Alkaline 1.5V 24шт/уп (DURACELL AA LR6 BL24),https://tiraet.com/catalog/jelektropitanie/batarei-akkumuljatornye/jelementy-pitanija/element_pitaniya_duracell_basic_aa_lr6_mn1500_alkaline_1_5v_24sht_up_duracell_aa_lr6_bl24/,2024-06-28,Мало,84971,399.6,NaN,T_электропитание,T_элементы_питания,T_батарейки_аккумуляторы,0
11676,Элемент питания Duracell BASIC AAA/LR3/MN2400 Alkaline 1.5V 24шт/уп (DURACELL BASIC AAA/LR3 BL24),https://tiraet.com/catalog/jelektropitanie/batarei-akkumuljatornye/jelementy-pitanija/element_pitaniya_duracell_basic_aaa_lr3_mn2400_alkaline_1_5v_24sht_up_duracell_basic_aaa_lr3_bl24/,2024-06-28,Мало,84972,400.3,NaN,T_электропитание,T_элементы_питания,T_батарейки_аккумуляторы,0
11677,"Батарея аккумуяторная (5809LYDD-ZJ) LI-ION 2000mAh, DC 7.4V/3A для термопринтеров POS-5809DD POS-5811DD",https://tiraet.com/catalog/jelektropitanie/batarei-akkumuljatornye/batareya_akkumuyatornaya_5809lydd_zj_li_ion_2000mah_dc_7_4v_3a_dlya_termoprinterov_pos_5809dd_pos_58/,2024-06-28,Много,82785,165.0,NaN,T_электропитание,T_элементы_питания,T_элементы_питания,0


In [23]:
len(data)

2

In [24]:
for i in df['_category'].unique():
    t = df[df['_category']==i]
    print(i)
    print(t['_subcategory'].unique())
    print('-'*100)

T_автотовары
['T_автомагнитола' 'T_автопылесос' 'T_авторегистратор' 'T_автохолодильник'
 'T_динамики' 'T_компрессоры' 'T_крепления_смартфонов/планшетов'
 'T_модуляторы_fm' 'T_навигаторы_gps']
----------------------------------------------------------------------------------------------------
T_безопасность
['T_видеонаблюдение' 'T_домофоны' 'T_монтаж_и_питание'
 'T_оборудование_poe' 'T_сигнализация_охранно_пожарная' 'T_скуд'
 'T_умный_дом']
----------------------------------------------------------------------------------------------------
T_бытовая_техника
['T_гигиена_здоровье_уход' 'T_климатическая_техника'
 'T_крупная_бытовая_техника' 'T_мелкая_бытовая_техника' 'T_телевизоры'
 'T_техника_для_кухни' 'T_часы_наручные']
----------------------------------------------------------------------------------------------------
T_для_дома
['T_бытовая_химия' 'T_запасные_части' 'T_инструменты' 'T_крепления'
 'T_кухонная_утварь' 'T_мебель' 'T_текстиль' 'T_хозяйственный_инвентарь']
-----------------

# Аналитика акционных товаров

In [25]:
brend = pd.read_excel('csv/brend.xlsx')

In [26]:
def lower(x):
    return x.lower()
brend['names'] = brend['name'].apply(lower)
brend_up = brend.name.unique()
brend_low = brend.names.unique()

Создаем список брендов

In [27]:
def brends(x):
    xx = x.replace('\xa0', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'
tqdm.pandas()
df['brend'] = df['title'].progress_apply(brends)

  0%|          | 0/11679 [00:00<?, ?it/s]

создаем столбец с брендами

In [28]:
df['action_sale'] = 0
df.loc[df['old_price'] > 0,'action_sale'] = 1

Создаем столбец с флагом акции

In [29]:
df['sale'] = (100 - ((df['price']*100)/df['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [30]:
def count_group(x):
    return df[df['_group']==x]['id'].count()    
tqdm.pandas()
df['count_group'] = df['_group'].progress_apply(count_group)

  0%|          | 0/11679 [00:00<?, ?it/s]

создание столбца с количеством товара в группе

In [31]:
def count_group_presence(x):
    return df[(df['availability'].isin(['Мало','Достаточно','Много']))&(df['_group']==x)]['id'].count()   
tqdm.pandas()
df['activ_count_group'] = df['_group'].progress_apply(count_group_presence)

  0%|          | 0/11679 [00:00<?, ?it/s]

создание столбца с количеством товара, который есть в наличии, в группе

In [32]:
def dif_price(x):    
    max_price = df[(df['availability'].isin(['Мало','Достаточно','Много']))&(df['_group']==x)]['price'].max()
    min_price = df[(df['availability'].isin(['Мало','Достаточно','Много']))&(df['_group']==x)]['price'].min() 
    return max_price / min_price
tqdm.pandas()
df['dif_price'] = df['_group'].progress_apply(dif_price)

  0%|          | 0/11679 [00:00<?, ?it/s]

создам столбец с отношением самой дорогой цены в группе к самой дешевой.

In [33]:
df['group_prise'] = df.groupby('_group')['price'].transform(lambda x: pd.cut(x, bins = 4, labels=["бюджет", "стандарт", 'дорогой', "элитный"]))

создание столбца с ценовой категорией товара

In [34]:
df = df[['url', 'id', '_category', '_subcategory', '_group', 'brend', 
         'title', 'date', 'price', 'old_price', 'action_sale', 'sale', 'group_prise', 'dif_price',
         'availability', 'count_group', 'activ_count_group', 'error_group']]

In [35]:
df['tir'] = 1

C:\Users\user\AppData\Local\Temp\ipykernel_12008\2466544301.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tir'] = 1


In [36]:
df = df.drop_duplicates().reset_index(drop=True)

In [37]:
df.to_csv('csv/tiraet_final_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False) 

In [38]:
df.head(2)

,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
0,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_sencor_sct_3018mr_fm_am_cd_mp3_wma_usb_aux_4_40w_iso_rca_1din/,78604,T_автотовары,T_автомагнитола,T_автомагнитола,sencor,"Автомагнитола Sencor SCT 3018MR (FM/AM, CD, MP3/WMA, USB, AUX, 4*40W, ISO/RCA, 1DIN)",2024-06-28,660.0,NaN,0,NaN,бюджет,3.331818,Мало,6,6,0,1
1,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_kenwood_kdc_130ug_fm_am_mp3_wav_wma_flac_cd_usb_aux_4_50w_iso_rca_1din_black/,78235,T_автотовары,T_автомагнитола,T_автомагнитола,kenwood,"Автомагнитола Kenwood KDC-130UG (FM/AM, MP3/WAV/WMA/FLAC, CD, USB, AUX, 4*50W, ISO/RCA, 1DIN) black",2024-06-28,1419.0,NaN,0,NaN,стандарт,3.331818,Мало,6,6,0,1


In [39]:
preprocessing(df)

таблица имеет следующий вид:


,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
0,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_sencor_sct_3018mr_fm_am_cd_mp3_wma_usb_aux_4_40w_iso_rca_1din/,78604,T_автотовары,T_автомагнитола,T_автомагнитола,sencor,"Автомагнитола Sencor SCT 3018MR (FM/AM, CD, MP3/WMA, USB, AUX, 4*40W, ISO/RCA, 1DIN)",2024-06-28,660.0,NaN,0,NaN,бюджет,3.331818,Мало,6,6,0,1
1,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_kenwood_kdc_130ug_fm_am_mp3_wav_wma_flac_cd_usb_aux_4_50w_iso_rca_1din_black/,78235,T_автотовары,T_автомагнитола,T_автомагнитола,kenwood,"Автомагнитола Kenwood KDC-130UG (FM/AM, MP3/WAV/WMA/FLAC, CD, USB, AUX, 4*50W, ISO/RCA, 1DIN) black",2024-06-28,1419.0,NaN,0,NaN,стандарт,3.331818,Мало,6,6,0,1
2,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_kenwood_kmm_bt209_fm_am_bt_usb_aux_rca_4_50w_iso_1din/,80185,T_автотовары,T_автомагнитола,T_автомагнитола,kenwood,"Автомагнитола Kenwood KMM-BT209 (FM/AM, BT, USB, AUX, RCA, 4*50W, ISO, 1DIN)",2024-06-28,1735.0,NaN,0,NaN,дорогой,3.331818,Достаточно,6,6,0,1
3,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_jvc_kdx_382bt_fm_am_bluetooth_usb_4_50w_iso_aux_1din/,80184,T_автотовары,T_автомагнитола,T_автомагнитола,jvc,"Автомагнитола JVC KDX-382BT (FM/AM, Bluetooth, USB, 4*50W, ISO, AUX, 1DIN)",2024-06-28,1815.0,NaN,0,NaN,элитный,3.331818,Мало,6,6,0,1
4,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_jvc_kdt_702bt_fm_am_bluetooth_cd_usb_4_50w_1din/,80183,T_автотовары,T_автомагнитола,T_автомагнитола,jvc,"Автомагнитола JVC KDT-702BT (FM/AM, Bluetooth, CD, USB, 4*50W, 1DIN)",2024-06-28,2099.0,NaN,0,NaN,элитный,3.331818,Достаточно,6,6,0,1


****************************************************************************************************
таблица имеет 11666 сторок и 19 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                      0
id                       0
_category                0
_subcategory             0
_group                   0
brend                    0
title                    0
date                     0
price                    2
old_price            10118
action_sale              0
sale                 10118
group_prise              2
dif_price                2
availability             0
count_group              0
activ_count_group        0
error_group              0
tir                      0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url                   0.0
id                    0.0
_category             0.0
_subcategory          0.0
_group                0.0
brend                 0.0
title                 0.0
date                  0.0
price                 0.0
old_price            86.7
action_sale           0.0
sale                 86.7
group_prise           0.0
dif_price             0.0
availability          0.0
count_group           0.0
activ_count_group     0.0
error_group           0.0
tir                   0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                          object
id                           object
_category                    object
_subcategory                 object
_group                       object
brend                        object
title                        object
date                 datetime64[ns]
price                       float64
old_price                   float64
action_sale                   int64
sale                        float64
group_prise                category
dif_price                   float64
availability                 object
count_group                   int64
activ_count_group             int64
error_group                   int64
tir                           int64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11666 entries, 0 to 11665
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   url                11666 non-null  object        
 1   id                 11666 non-null  object        
 2   _category          11666 non-null  object        
 3   _subcategory       11666 non-null  object        
 4   _group             11666 non-null  object        
 5   brend              11666 non-null  object        
 6   title              11666 non-null  object        
 7   date               11666 non-null  datetime64[ns]
 8   price              11664 non-null  float64   

None

# Тесты

In [5]:
url = ['https://tiraet.com/catalog/PHoto--i-video/?PAGEN_1=']

section-gallery-wrapper__item _active

In [42]:
reqx = requests.get('https://tiraet.com/catalog/rashodnye-materialy/?PAGEN_1=3')
reqx

<Response [200]>

In [43]:
soup = bs(reqx.text, 'html.parser')
soup

<!DOCTYPE html>

<html lang="ru" xml:lang="ru" xmlns="http://www.w3.org/1999/xhtml">
<head><link href="https://tiraet.com/catalog/rashodnye-materialy/?PAGEN_1=4" rel="next"/><link href="https://tiraet.com/catalog/rashodnye-materialy/?PAGEN_1=2" rel="prev"/><link href="https://tiraet.com/catalog/rashodnye-materialy/" rel="canonical"/>
<title>РАСХОДНЫЕ МАТЕРИАЛЫ купить в магазине Тираэт в Тирасполе, Бендерах, Рыбнице, Дубоссарах</title>
<meta content="initial-scale=1.0, width=device-width, maximum-scale=1" name="viewport"/>
<meta content="true" name="HandheldFriendly"/>
<meta content="yes" name="yes"/>
<meta content="black" name="apple-mobile-web-app-status-bar-style"/>
<meta content="SKYPE_TOOLBAR_PARSER_COMPATIBLE" name="SKYPE_TOOLBAR"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="компьютер, ноутбук, бытовая техника, TV, XBOX, playstation, PS, game купить по самой низкой цене" name="keywords"/>
<meta content="РАСХОДНЫЕ МАТЕРИАЛЫ купить в магазине

In [31]:
element2 = soup.findAll('div', class_='inner_wrap TYPE_3')

In [32]:
element2 = soup.find('span', class_="section-gallery-wrapper__item _active")

In [46]:
element = soup.findAll('div', class_='image_wrapper_block js-notice-block__image')

In [77]:
element[3].find('span',class_="section-gallery-wrapper__item _active").text

'\n\n\n'

In [14]:
element2[0].text

KeyError: 0

In [15]:
for i in element2:
    print(element2[i].text)


KeyError: '\n'

In [ ]:
type(element2)